<a href="https://colab.research.google.com/github/AnnaK8090/CIND-820_Big-Data-Analytics-Project/blob/main/CIND_820_Big_Data_Analytics_Project_2_Collaborative_Filtering_Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd      

In [25]:
masterDF = pd.read_csv('MasterDF.csv', on_bad_lines='skip')

In [26]:
masterDF.shape

(113210, 40)

In [27]:
# Order ID might have more than 1 review (given on the same or different date), so we will aggregate orders by order_id and choose max review score:
masterDF_grouped = masterDF.groupby(['order_id','customer_unique_id','product_id','product_category_name_english'])['review_score'].max()
masterDF_grouped = masterDF_grouped.reset_index()
masterDF_grouped.shape

(98091, 5)

In [28]:
masterDF_grouped.to_csv ('MasterDF_grouped.csv', index = None, header=True)

In [29]:
masterDF_grouped.head()

,order_id,customer_unique_id,product_id,product_category_name_english,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,871766c5855e863f6eccc05f988b23cb,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,5
1,00018f77f2f0320c557190d7a144bdd3,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,pet_shop,4
2,000229ec398224ef6ca0657da4fc703e,3818d81c6709e39d06b2738a8d3a2474,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,5
3,00024acbcdf0a6daa1e931b038114c75,af861d436cfc08b2c2ddefd0ba074622,7634da152a4610f1595efa32f14722fc,perfumery,4
4,00042b26cf59d7ce69dfabb4e55b4fd9,64b576fb70d441e8f1b2d7d446e483c5,ac6c3623068f30de03045865e4e10089,garden_tools,5


In [31]:
#dataframe with those orders that have at least 2 product per customer
new = masterDF_grouped.groupby(['order_id'])['product_category_name_english'].agg('count').reset_index()
display(new)

,order_id,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
94483,fffc94f6ce00a00581880bf54a75a037,1
94484,fffcd46ef2263f404302a634eb57f7eb,1
94485,fffce4705a9662cd70adb13d4a31832d,1
94486,fffe18544ffabc95dfada21779c9644f,1


In [32]:
ProductsPerCustomer = masterDF_grouped.groupby(['customer_unique_id'])['product_id'].agg('count').reset_index()
ProductsPerCustomer.shape

(91481, 2)

In [33]:
ProductsGreater1PerCustomer = ProductsPerCustomer.loc[ProductsPerCustomer['product_id'] >1]
ProductsGreater1PerCustomer.shape

(5471, 2)

In [34]:
ProductsGreater1PerCustomer.head()

,customer_unique_id,product_id
7,00053a61a98854899e70ed204dd4bafe,2
18,000de6019bb59f34c099a907c151d855,2
22,000fbf0473c10fc1ab6f8d2d286ce20c,2
35,001926cef41060fae572e2e7b30bd2a4,2
36,001928b561575b2821c92254a2327d06,2


In [35]:
result = masterDF_grouped[(masterDF_grouped.customer_unique_id.isin(ProductsGreater1PerCustomer.customer_unique_id))]
result.head()

,order_id,customer_unique_id,product_id,product_category_name_english,review_score
1,00018f77f2f0320c557190d7a144bdd3,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,pet_shop,4
35,0017afd5076e074a48f1f1a4c7bac9c5,98758d88bf4b8eef1372ddee45d63178,fe59a1e006df3ac42bf0ceb876d70969,computers_accessories,1
48,0020a222f55eb79a372d0efee3cca688,87ae4c644c15d9c6b6f826dfec33b340,e19ddcc85537b41f22116c8d5425ef46,furniture_decor,5
71,002f98c0f7efd42638ed6100ca699b42,2e875ea57961ad115cec13fef0920ae6,880be32f4db1d9f6e2bec38fb6ac23ab,toys,5
72,002f98c0f7efd42638ed6100ca699b42,2e875ea57961ad115cec13fef0920ae6,d41dc2f2979f52d75d78714b378d4068,consoles_games,5


In [36]:
result.shape

(12081, 5)

In [37]:
import progressbar as pb
import random

In [38]:
#Function to convert dataframe into array 

c_data = pd.DataFrame()
products = []
c_data['order_id'] = 0
for product in pb.progressbar(result['product_id']):
    if product not in products:
        products.append(product)
        c_data[product] = 0
users = []
for user in pb.progressbar(result['customer_unique_id']):
    if user not in users:
        users.append(user)
        append_dic = {'order_id':user} 
        for column in c_data.columns:
            if column != 'order_id':
                append_dic[column] = 0
        c_data = c_data.append(pd.DataFrame([append_dic]))
c_data = c_data.set_index('order_id')

N/A% (0 of 12081) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if __name__ == '__main__':
100% (12081 of 12081) |##################| Elapsed Time: 0:00:08 Time:  0:00:08
100% (12081 of 12081) |##################| Elapsed Time: 1:10:35 Time:  1:10:35


In [39]:
result.reset_index(inplace = True,drop = True)

In [40]:
for index in pb.progressbar(range(len(result)-1)):
    c_data.loc[result['customer_unique_id'][index],result['product_id'][index]] = result['review_score'][index]


100% (12080 of 12080) |##################| Elapsed Time: 0:00:01 Time:  0:00:01


In [42]:
c_data.to_csv ('c_data.csv', index = None, header=True)

In [43]:
c_data.head()

,e5f2d52b802189ee658865ca93d83a8f,fe59a1e006df3ac42bf0ceb876d70969,e19ddcc85537b41f22116c8d5425ef46,880be32f4db1d9f6e2bec38fb6ac23ab,d41dc2f2979f52d75d78714b378d4068,13944d17b257432717fd260e69853140,1f9799a175f50c9fa725984775cac5c5,8a6187b2665118d5095f99a25fd7ba7a,4c3ae5db49258df0784827bdacf3b396,fb7a100ec8c7b34f60cec22b1a9a10e0,...,e86b81dcac341ea01df0260077cdf082,8ee15ce472d128e8ea95d9a24c2d397c,bd0ac51dc93e62c4dbe6ca9d70a9b311,bd6e8cf9fe4122c385da2bcb9f979d5d,3321ad579f19476d0d668f726f8dffec,241e398aacc909372622952b2ec6f954,7c1043bb5837db0c6bc1953419a18628,102cdda7fcf3cb71b5f9d6d2a3e56b58,b10ecf8e33aaaea419a9fa860ea80fb5,75f6a4f019ec1322758d53b2fee2cc12
order_id,,,,,,,,,,,,,,,,,,,,,
eb28e67c4c0b83846050ddfb8a35d051,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98758d88bf4b8eef1372ddee45d63178,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87ae4c644c15d9c6b6f826dfec33b340,0,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2e875ea57961ad115cec13fef0920ae6,0,0,0,5,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3c857a6f7828bfb70fb712e2393cfd1b,0,0,0,0,0,5,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
c_data.shape

(5471, 7602)

In [45]:
# Matrix factorization function:
def matrix_factorization(R, P, Q, K, steps=1000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in pb.progressbar(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T

In [46]:
#VERY IMPORTANT to transform dataframe to array
R = np.array(c_data)

In [47]:
N = len(R)
print(N)

5471


In [48]:
R.reset_index(inplace = True,drop = True)

AttributeError: ignored

In [49]:
M = len(R[0])
print(M)

7602


In [ ]:
K = 2

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)

N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--